# (Einfache) Simulierung einer Turing Bombe
Dieses Notebook dient dazu die Funktionsweise der Turing Bombe zu verstehen. Der Einfachheit halber, haben wir in diesem Beispiel darauf verzichtet ein Steckerbrett zu verwenden. 
Es ist aber jederzeit möglich dieses zu ergänzen (Für die neugierigen Leser).

Import der Module (notwendig)

In [ ]:
from sagEnigma import Enigma
import threading
from queue import Queue

Implementierung einer Enigma 1 ohne Steckerbrett. NB: Wir haben uns entschieden Multithreading zu nutzen, hierfür wird eine Queue mit einzelnen Aufgaben erstellt. Diese Aufgaben stellen das parallele Testen der Eingma Konfigurationen da.

In [5]:
class Bombe:
    def __init__(self, max_wheel_val, max_position_val, max_reflector_val, crib, encrypted_word):
        self.max_wheel_val = max_wheel_val
        self.max_position_val = max_position_val
        self.max_reflector_val = max_reflector_val
        self.crib = crib
        self.encrypted_word = encrypted_word
        self.tasks = Queue()
        self.fill_tasks()

    def fill_tasks(self):
        for wheel1 in range(1, self.max_wheel_val):
            for wheel2 in range(1, self.max_wheel_val):
                for wheel3 in range(1, self.max_wheel_val):
                    for position1 in range(1, self.max_position_val):
                        for position2 in range(1, self.max_position_val):
                            for position3 in range(1, self.max_position_val):
                                for offset1 in range(1, self.max_position_val):
                                    for offset2 in range(1, self.max_position_val):
                                        for offset3 in range(1, self.max_position_val):
                                            for reflector in range(1, self.max_reflector_val):
                                                self.tasks.put(
                                                    [wheel1, wheel2, wheel3, position1, position2, position3,
                                                     offset1, offset2, offset3, reflector])
    def get_se_germans(self):
        try:
            while not self.tasks.empty():
                enigma_config = self.tasks.get()
                wheel_1, wheel_2, wheel_3, position_1, position_2, position_3, offset1, offset2, offset3, reflector = enigma_config
                threaded_enigma = Enigma.create([wheel_1, wheel_2, wheel_3], [position_1, position_2, position_3],
                                                [offset1, offset2, offset3], reflector)
                decrypted_text = threaded_enigma.en_de_crypt(self.encrypted_word)
                if decrypted_text == self.crib:
                    print(f"Decrypted Message: {decrypted_text}")
                    print(f"One possible setting of the Day: {enigma_config} go get those Fritzes!\n")
                self.tasks.task_done()
        except Exception as e:
            print(e)

    def crack(self):
        for _ in range(16):
            threading.Thread(target=self.get_se_germans).start()
        self.tasks.join()

Entschlüsselung: Mehrere Enigmas werden erstellt und probieren ob ihre Konfig das gewünschte Match aus bekanntem Wort und Cypher-Text enhält. Falls ja, wird die Config zurück gegeben.

Korrekter max_wheel_val = 6, max_position_val = 27, max_ukw_val = 4
Um schneller testen zu können, ob die Bombe funktioniert haben wir die Werte nach unten gesetzt. Wer mehr Zeit hat, darf sie gerne auf den korrekten Wert setzen.

In [6]:
bomber = Bombe(3,3,3, "HALLO", "fuytv")
bomber.crack()

Decrypted Message: HALLODecrypted Message: HALLO
One possible setting of the Day: [1, 2, 2, 2, 1, 1, 2, 2, 2, 2] go get those Fritzes!


One possible setting of the Day: [1, 2, 2, 1, 1, 1, 1, 2, 2, 2] go get those Fritzes!

